In [2]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
print(spark.version)


TypeError: 'JavaPackage' object is not callable

In [7]:
orders_df = spark.read.csv("C:/data/orders_sh.csv", header=True, inferSchema=True)

In [8]:
orders_df.createOrReplaceTempView("orders_tmp")

In [9]:
spark.sql("select * from orders_tmp").show(5)

+--------+----------+-----------+---------------+
|order_id|order_date|customer_id|   order_status|
+--------+----------+-----------+---------------+
|       1|   00:00.0|      11599|         CLOSED|
|       2|   00:00.0|        256|PENDING_PAYMENT|
|       3|   00:00.0|      12111|       COMPLETE|
|       4|   00:00.0|       8827|         CLOSED|
|       5|   00:00.0|      11318|       COMPLETE|
+--------+----------+-----------+---------------+
only showing top 5 rows



In [12]:
spark.sql("create database if not exists spark_sql_db")

DataFrame[]

In [13]:
spark.sql("show databases").show()

+------------+
|   namespace|
+------------+
|     default|
|spark_sql_db|
+------------+



In [16]:
spark.sql("create table if not exists spark_sql_db.orders_managed as select * from orders_tmp")

Py4JJavaError: An error occurred while calling o27.sql.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalog.validateTableLocation(SessionCatalog.scala:413)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalog.createTable(SessionCatalog.scala:400)
	at org.apache.spark.sql.hive.execution.Crea